In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder


from tqdm.notebook import tqdm

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, BaggingClassifier

# ustawia domyślną wielkość wykresów
plt.rcParams['figure.figsize'] = (12,8)
# to samo tylko dla tekstu
plt.rcParams['font.size'] = 16

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# wczytanie danych
train = pd.read_csv('data/airline_passenger_satisfaction/train.csv')
test = pd.read_csv('data/airline_passenger_satisfaction/test.csv')

## Obróbka danych

In [3]:
train.columns

Index(['Unnamed: 0', 'id', 'Gender', 'Customer Type', 'Age', 'Type of Travel',
       'Class', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [4]:
train.isnull().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction    

In [5]:
test.isnull().sum()

Unnamed: 0                            0
id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             83
satisfaction                          0


In [6]:
train = train.dropna()
test = test.dropna()

In [7]:
ord_enc = OrdinalEncoder()

train = train.drop(columns = ['Unnamed: 0', 'id', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness'])

train["Gender"] = ord_enc.fit_transform(train[["Gender"]])
train["Customer Type"] = ord_enc.fit_transform(train[["Customer Type"]])
train["Type of Travel"] = ord_enc.fit_transform(train[["Type of Travel"]])
train["satisfaction"] = ord_enc.fit_transform(train[["satisfaction"]])
train["Class"] = ord_enc.fit_transform(train[["Class"]])




test = test.drop(columns = ['Unnamed: 0', 'id', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness'])

test["Gender"] = ord_enc.fit_transform(test[["Gender"]])
test["Customer Type"] = ord_enc.fit_transform(test[["Customer Type"]])
test["Type of Travel"] = ord_enc.fit_transform(test[["Type of Travel"]])
test["satisfaction"] = ord_enc.fit_transform(test[["satisfaction"]])
test["Class"] = ord_enc.fit_transform(test[["Class"]])

In [8]:
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1.0,0.0,13,1.0,2.0,460,25,18.0,0.0
1,1.0,1.0,25,0.0,0.0,235,1,6.0,0.0
2,0.0,0.0,26,0.0,0.0,1142,0,0.0,1.0
3,0.0,0.0,25,0.0,0.0,562,11,9.0,0.0
4,1.0,0.0,61,0.0,0.0,214,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
103899,0.0,1.0,23,0.0,1.0,192,3,0.0,0.0
103900,1.0,0.0,49,0.0,0.0,2347,0,0.0,1.0
103901,1.0,1.0,30,0.0,0.0,1995,7,14.0,0.0
103902,0.0,1.0,22,0.0,1.0,1000,0,0.0,0.0


## Podział danych

In [9]:
X_train = train.drop(columns=['satisfaction'])
X_test = test.drop(columns=['satisfaction'])

y_train = train.satisfaction
y_test = test.satisfaction

In [10]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=420, train_size=0.66)

In [11]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

print(X_val.shape)
print(y_val.shape)

(103594, 8)
(103594,)
(17089, 8)
(17089,)
(8804, 8)
(8804,)


## Random Forest

In [12]:
rf = RandomForestClassifier()

In [13]:
rf.fit(X_train, y_train).score(X_test, y_test)

0.7605477207560419

In [14]:
#przetestujmy jak działa to dla różnych parametrów
scores = []
for i in [1, 2, 4, 8, 16, 32, 64, 128]:
    for j in  [1, 2, 4, 8, 16, 32, 64, 128]:
        for k in  [2, 4, 8, 16, 32, 64, 128]:
            best = [0,0,0,0]
            rf = RandomForestClassifier(max_depth = i, n_estimators = j, max_leaf_nodes = k)
            score = rf.fit(X_train, y_train).score(X_test, y_test)
            print (f"max_depth: {i},  n_estimators: {j}, max_leaf_nodes: {k}, score: {score}")
            if (score > best[3]):
                best = [i, j, k, score]
            scores.append(score)
            

max_depth: 1,  n_estimators: 1, max_leaf_nodes: 2, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, score: 0.5603604657967113
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 8, score: 0.5603604657967113
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, score: 0.683129498507812
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 32, score: 0.5811925800222365
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 64, score: 0.7476739423020656
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 128, score: 0.7476739423020656
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 2, score: 0.7476739423020656
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 4, score: 0.7636491310199544
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 8, score: 0.7636491310199544
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 16, score: 0.7476739423020656
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 32, score: 0.6048920358125109
max_depth: 1,  n_estimators: 2, max_leaf_nodes: 64, score: 0.747673942

max_depth: 2,  n_estimators: 128, max_leaf_nodes: 2, score: 0.774942945754579
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 4, score: 0.7684475393528001
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 8, score: 0.7750599801041606
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 16, score: 0.7686230908771724
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 32, score: 0.7734800163848089
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 64, score: 0.7733044648604366
max_depth: 2,  n_estimators: 128, max_leaf_nodes: 128, score: 0.7747088770554158
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 2, score: 0.5603604657967113
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 4, score: 0.742290362221312
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 8, score: 0.7790976651647259
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 16, score: 0.7521797647609574
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 32, score: 0.7341564749253906
max_depth: 4,  n_estimators: 1, max_leaf_nodes: 64, score

max_depth: 8,  n_estimators: 64, max_leaf_nodes: 2, score: 0.7738311194335538
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 4, score: 0.7751770144537422
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 8, score: 0.7785124934168178
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 16, score: 0.7810287319328223
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 32, score: 0.7914447890455849
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 64, score: 0.7940780619111709
max_depth: 8,  n_estimators: 64, max_leaf_nodes: 128, score: 0.7974720580490374
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 2, score: 0.7656972321376324
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 4, score: 0.7742992568318802
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 8, score: 0.7786880449411903
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 16, score: 0.7861782433144128
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 32, score: 0.7915033062203757
max_depth: 8,  n_estimators: 128, max_leaf_nodes: 64

max_depth: 32,  n_estimators: 16, max_leaf_nodes: 128, score: 0.7959506115044765
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 2, score: 0.7505998010416057
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 4, score: 0.7729533618116917
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 8, score: 0.7774591842705835
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 16, score: 0.7882848616068816
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 32, score: 0.7908596172976768
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 64, score: 0.7935514073380537
max_depth: 32,  n_estimators: 32, max_leaf_nodes: 128, score: 0.798174264146527
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 2, score: 0.7682719878284276
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 4, score: 0.7745918427058341
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 8, score: 0.7817894552051027
max_depth: 32,  n_estimators: 64, max_leaf_nodes: 16, score: 0.7868804494119024
max_depth: 32,  n_estimators: 64, max_leaf_no

max_depth: 128,  n_estimators: 8, max_leaf_nodes: 64, score: 0.7916203405699573
max_depth: 128,  n_estimators: 8, max_leaf_nodes: 128, score: 0.7885189303060448
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 2, score: 0.7653461290888876
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 4, score: 0.7707882263444321
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 8, score: 0.7783369418924454
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 16, score: 0.7871730352858564
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 32, score: 0.7926151325414009
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 64, score: 0.7939610275615894
max_depth: 128,  n_estimators: 16, max_leaf_nodes: 128, score: 0.7978231610977822
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 2, score: 0.7606647551056235
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 4, score: 0.7765814266487214
max_depth: 128,  n_estimators: 32, max_leaf_nodes: 8, score: 0.780209491485751
max_depth: 128,  n_estimators: 32, 

In [16]:
print (f"mediana: {np.median(scores)}")
print (f"srednia: {np.mean(scores)}")
print (f"najlepszy: max_depth: {best[0]}, n_estimators: {best[1]}, max_leaf_nodes: {best[2]}, score: {best[3]}")

mediana: 0.7747381356428111
srednia: 0.7652343194870552
najlepszy: max_depth: 128, n_estimators: 128, max_leaf_nodes: 128, score: 0.7975305752238282


## XGBoost

In [18]:
xgb_cls = xgb.XGBClassifier(objective="binary:logistic", seed = 42, use_label_encoder=False)

In [19]:
xgb_cls.fit(X_train, y_train, verbose=True, early_stopping_rounds=5, eval_metric="error", eval_set=[(X_val, y_val)])

[0]	validation_0-error:0.19945
[1]	validation_0-error:0.19764
[2]	validation_0-error:0.19798
[3]	validation_0-error:0.19752
[4]	validation_0-error:0.19650
[5]	validation_0-error:0.19616


/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[6]	validation_0-error:0.19662
[7]	validation_0-error:0.19616
[8]	validation_0-error:0.19548
[9]	validation_0-error:0.19662
[10]	validation_0-error:0.19559
[11]	validation_0-error:0.19605
[12]	validation_0-error:0.19593
[13]	validation_0-error:0.19503
[14]	validation_0-error:0.19480
[15]	validation_0-error:0.19525
[16]	validation_0-error:0.19559
[17]	validation_0-error:0.19571
[18]	validation_0-error:0.19559


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=10,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [20]:
xgb_cls.score(X_test, y_test)

0.8009245713616947

In [ ]:
#przetestujmy jak działa to dla różnych parametrów
scores = []
for i in [1, 4, 16, 64, 128]:
    for j in  [1, 4, 16, 64, 128]:
        for k in  [4, 16, 64, 128]:
            for l in [1, 4, 16, 64, 128]:
                best = [0,0,0,0,0]
                xgb_cls = xgb.XGBClassifier(objective="binary:logistic", 
                                       seed = 42, 
                                       use_label_encoder=False, 
                                       max_depth = i, 
                                       n_estimators = j, 
                                       max_leaf_nodes = k, 
                                       max_leaves = l)

                score = xgb_cls.fit(X_train, y_train, verbose = False).score(X_test, y_test)
                print (f"max_depth: {i},  n_estimators: {j}, max_leaf_nodes: {k}, max_leaves: {l}, score: {score}")
                if (score > best[4]):
                    best = [i, j, k, l, score]
                scores.append(score)

[22:58:51] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:58:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 4, max_leaves: 1, score: 0.7476739423020656
[22:58:51] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pan

[22:58:51] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:58:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 16, max_leaves: 1, score: 0.7476739423020656
[22:58:51] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but gettin

/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pan

[22:58:52] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:58:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
max_depth: 1,  n_estimators: 1, max_leaf_nodes: 64, max_leaves: 1, score: 0.7476739423020656
[22:58:52] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but gettin

...

[23:40:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
max_depth: 64,  n_estimators: 128, max_leaf_nodes: 4, max_leaves: 128, score: 0.7620106501258119
[23:41:59] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[23:41:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
max_depth: 64,  n_estimators: 128, max_leaf_nodes: 16, max_leaves: 1, score: 0.7620106501258119
[23:43:21] WARNING: ../src/learner.cc:576: 
Parameters: { "max_leaf_nodes" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




/Users/adammajczyk/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[23:43:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
print (f"mediana: {np.median(scores)}")
print (f"srednia: {np.mean(scores)}")
print (f"najlepszy: max_depth: {best[0]}, n_estimators: {best[1]}, max_leaf_nodes: {best[2]}, max_leaves: {best[3]}, score: {best[4]}")

## Wnioski 
* Z XGBoost jest lepszy z parametrami domyślnymi niż Random Forest (także z domyślnymi).

* XGBoost okazuje się być średnio lepszy niż Random Forest.

* W przypadku obu modeli dobranie większych parametrów pozytywnie wpływa na score (wynik best)